<a href="https://colab.research.google.com/github/amoghgopadi/wav2vec2-xlsr-kannada/blob/main/Fine_Tune_XLSR_Wav2Vec2_on_Kannada_ASR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-tuning XLSR-Wav2Vec2 for Multi-Lingual ASR with 🤗 Transformers**

Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.  Soon after the superior performance of Wav2Vec2 was demonstrated on the English ASR dataset LibriSpeech, *Facebook AI* presented XLSR-Wav2Vec2 (click [here](https://arxiv.org/abs/2006.13979)). XLSR stands for *cross-lingual  speech representations* and refers to XLSR-Wav2Vec2`s ability to learn speech representations that are useful across multiple languages.

Similar to Wav2Vec2, XLSR-Wav2Vec2 learns powerful speech representations from hundreds of thousands of hours of speech in more than 50 languages of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/xlsr_wav2vec2.png)

The authors show for the first time that massively pretraining an ASR model on cross-lingual unlabeled speech data, followed by language-specific fine-tuning on very little labeled data achieves state-of-the-art results. See Table 1-5 of the official [paper](https://arxiv.org/pdf/2006.13979.pdf).

In this notebook, we will give an in-detail explanation of how XLSR-Wav2Vec2's pretrained checkpoint can be fine-tuned on a low-resource ASR dataset of any language. Note that in this notebook, we will fine-tune XLSR-Wav2Vec2 without making use of a language model. It is much simpler and more efficient to use XLSR-Wav2Vec2 without a language model, but better results can be achieved by including a language model. 

For demonstration purposes, we fine-tune the [wav2vec2-large-xlsr-53](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) on the low resource Kannada ASR multi-speaker dataset of [OpenSLR](http://openslr.org/79/).

XLSR-Wav2Vec2 is fine-tuned using Connectionist Temporal Classification (CTC), which is an algorithm that is used to train neural networks for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition. 

I highly recommend reading the blog post [Sequence Modeling with CTC (2017)](https://distill.pub/2017/ctc/) very well-written blog post by Awni Hannun.

Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `torchaudio` and `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [ ]:
!nvidia-smi

Sat Mar 27 06:25:03 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 450.51.05    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100S-PCI...  Off  | 00000000:00:0A.0 Off |                    0 |
| N/A   35C    P0    25W / 250W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa
!pip install jiwer




---

${}^1$ In the [paper](https://arxiv.org/pdf/2006.13979.pdf), the model was evaluated using the phoneme error rate (PER), but by far the most common metric in ASR is the word error rate (WER). To keep this notebook as general as possible we decided to evaluate the model using WER.

## Prepare Data, Tokenizer, Feature Extractor

ASR models transcribe speech to text, which means that we both need a feature extractor that processes the speech signal to the model's input format, *e.g.* a feature vector, and a tokenizer that processes the model's output format to text. 

In 🤗 Transformers, the XLSR-Wav2Vec2 model is thus accompanied by both a tokenizer, called [Wav2Vec2CTCTokenizer](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2ctctokenizer), and a feature extractor, called [Wav2Vec2FeatureExtractor](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#wav2vec2featureextractor).

Let's start by creating the tokenizer responsible for decoding the model's predictions.

### Create Wav2Vec2CTCTokenizer

The [pretrained Wav2Vec2 checkpoint]( ) maps the speech signal to a sequence of context representations as illustrated in the figure above. A fine-tuned XLSR-Wav2Vec2 checkpoint needs to map this sequence of context representations to its corresponding transcription so that a linear layer has to be added on top of the transformer block (shown in yellow). This linear layer is used to classifies each context representation to a token class analogous how, *e.g.*, after pretraining a linear layer is added on top of BERT's embeddings for further classification - *cf.* with *"BERT"* section of this [blog post](https://huggingface.co/blog/warm-starting-encoder-decoder).

The output size of this layer corresponds to the number of tokens in the vocabulary, which does **not** depend on XLSR-Wav2Vec2's pretraining task, but only on the labeled dataset used for fine-tuning. So in the first step, we will take a look at Common Voice and define a vocabulary based on the dataset's transcriptions.

First, download the dataset to your drive and unzip it. Then, load the files and make a dataset using 🤗 Datasets' simple API.

In [ ]:
import os

In [ ]:
import pandas as pd

In [ ]:
#!wget 'https://openslr.magicdatatech.com/resources/79/kn_in_male.zip' -P '/content/gdrive/MyDrive/Colab Notebooks/HuggingFace/Test2'

In [ ]:
#!wget 'https://openslr.magicdatatech.com/resources/79/kn_in_female.zip' -P '/content/gdrive/MyDrive/Colab Notebooks/HuggingFace/Test2'

In [ ]:
#!unzip 'container_0/kn_in_male.zip' -d 'container_0/kn_male'

In [ ]:
#!unzip 'container_0/kn_in_female.zip' -d 'container_0/kn_female'

In [ ]:
path_to_dir = "container_0"
path_to_dir_male = "container_0/kn_male"
path_to_dir_female = "container_0/kn_female"

In [ ]:
paths = []
texts = []

In [ ]:
line_df_male = pd.read_csv(os.path.join(path_to_dir,'line_index_male.tsv'),sep="\t", header=None)
fils = os.listdir(path_to_dir_male)
for fil in fils:
    if '.wav' in fil:
        paths.append(os.path.join(path_to_dir_male,fil))
        texts.append(line_df_male[line_df_male[0]==fil.split('.')[0]][1].values[0])

In [ ]:
line_df_female = pd.read_csv(os.path.join(path_to_dir,'line_index_female.tsv'),sep="\t", header=None)
fils = os.listdir(path_to_dir_female)
for fil in fils:
    if '.wav' in fil:
        paths.append(os.path.join(path_to_dir_female,fil))
        texts.append(line_df_female[line_df_female[0]==fil.split('.')[0]][1].values[0])

In [ ]:
from datasets import Dataset
data = Dataset.from_dict({"path":paths,"text":texts})

Shuffle the dataset for randomness.

In [ ]:
data = data.shuffle(seed=42)

Split the dataset into train and test.

In [ ]:
dataset = data.train_test_split(test_size=0.1,seed=42)

Dataset({
    features: ['path', 'text'],
    num_rows: 4400
})

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['path', 'text'],
        num_rows: 3960
    })
    test: Dataset({
        features: ['path', 'text'],
        num_rows: 440
    })
})

In [ ]:
train_dataset = dataset['train']
test_dataset = dataset['test']

Let's write a short function to display some random samples of the dataset and run it a couple of times to get a feeling for the transcriptions.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_dataset.remove_columns(["path"]), num_examples=20)

,text
0,ತತ್ತ್ವಗಳನ್ನು ಸಂಖ್ಯಾನ ಮಾಡುವ ಪದ್ಧತಿ ಈ ನಿಕಾಯದಲ್ಲಿ ಆರಂಭವಾಗಿ ಅಭಿದಮ್ಮಪಿಟಕದಲ್ಲಿ ಬೆಳೆಯಿತು.
1,ಈ ರೀತಿ ಕಮಲಕ್ಕೆ ಹೋಲಿಸುವುದು ಆಯುರ್ವೇದ ಗ್ರಂಥಗಳಲ್ಲಿ ಸಾಮಾನ್ಯ
2,ವ್ಯವಸಾಯ ಹದಿನೈದು ದಿನಗಳಲ್ಲಿ ಮುಗಿಯಬೇಕು.
3,ಈ ಪ್ರಸ್ತಾಪವನ್ನು ಅವಿರೋಧವಾಗಿ ನೂರು ಮಹಿಳೆಯರು ಹದಿನೇಳು ದೇಶಗಳಿಂದ ಸಹಕರಿಸಿದರು
4,ಈ ಉದ್ಯಮವು ಪರಿಸರದ ಮೇಲೆ ಮಾರಕ ಪರಿಣಾಮವನ್ನುಂಟು ಮಾಡುತಿದೆ.
5,ಬಸವಣ್ಣನವರ ವಚನ ಸದಾಚಾರವೇ ಅಂತಃಶುದ್ಧಿಯ ಅಡಿಗಲ್ಲೆಂಬ ವಿಷಯವನ್ನು ಸ್ಪಷ್ಟಪಡಿಸುತ್ತದೆ.
6,ಭೂಮಿಯ ವಿದ್ಯುತ್ಕಾಂತೀಯ ಕ್ಷೇತ್ರವು ಪೃಥ್ವಿಯ ನಿಕಟವರ್ತಿ ಕಕ್ಷೆಯಲ್ಲಿನ ಸೂರ್ಯನ ಮತ್ತು ಇತರ ವಿಕಿರಣಗಳಿಂದ ಬಹುಪಾಲು ವಾಯುಮಂಡಲದಷ್ಟೇ ಸುರಕ್ಷತೆಯನ್ನು ಒದಗಿಸುತ್ತದೆ
7,ಸೂಕ್ಷ್ಮ ಗುರುತ್ವದಲ್ಲಿ ನಡೆಸುವ ಸ್ರವಿಯ ಭೌತಶಾಸ್ತ್ರದ ಪರೀಕ್ಷೆಯು ಒಳಗೊಂಡಿದೆ
8,ಮುನ್ನೂರು ಗಜಗಳಲ್ಲಿ
9,ಪ್ರತಿಯೊಂದು ಸಮಾಜವಾದಿ ಪಕ್ಷವೂ ಯುದ್ಧನಿವಾರಣೆಯ ಕಾರ್ಯದಲ್ಲಿ ಆಸಕ್ತಿಗೊಂಡಿತು.


We can see that the transcriptions contain some special characters, such as `,.?!;:`. Without a language model, it is much harder to classify speech chunks to such special characters because they don't really correspond to a characteristic sound unit. *E.g.*, the letter `"s"` has a more or less clear sound, whereas the special character `"."` does not.
Also in order to understand the meaning of a speech signal, it is usually not necessary to include special characters in the transcription.

In addition, we normalize the text to only have lower case letters and append a word separator token at the end.

In [ ]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\–\…]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
    return batch

In [ ]:
train_dataset = train_dataset.map(remove_special_characters)
test_dataset = test_dataset.map(remove_special_characters)

In [ ]:
show_random_elements(test_dataset.remove_columns(["path"]))

,text
0,ಸಾಫ್ಟ್ ವೇರ್ ಅಂತರಿಕ್ಷಯಾನದ ಬಳಕೆಗಾಗಿ ಕಂಪ್ಯೂಟರ್ ಸಾಫ್ಟ್ ವೇರ್ ನ ನಿರ್ದಿಷ್ಟ ವಿಶ್ಲೇಷಣೆ ಮಾಡಲಾಗುತ್ತದೆ
1,ಇನ್ನೂರು ಗಜಗಳಲ್ಲಿ
2,ಸಾಮಾನ್ಯವಾಗಿ ಅಂತರರಾಷ್ಟ್ರೀಯ ಹಣಕಾಸು ನಿಧಿ ಹಾಗೂ ಅದರ ಬೆಂಬಲಿಗರು ವಿತ್ತೀಯ ನಿಯಂತ್ರಣವನ್ನು ಆಚರಣೆಗೆ ತರಬೇಕೆಂದು ಹೇಳುತ್ತಾರೆ
3,ಇದಕ್ಕೆ ಸಂಬಂಧಪಟ್ಟ ವಿಷಯಗಳು ಘಟಕಗಳ ಮಾಲೀಕತ್ವ
4,ಆದರೆ ಇದರಿಂದ ಯಾವುದಕ್ಕೂ ತೊಂದರೆಯುಂಟಾಗಲಿಲ್ಲ
5,ಈ ಎರಡೂ ವ್ಯವಹಾರಗಳ ನಡುವೆ ಅನೇಕ ವಿಷಯಗಳಲ್ಲಿ ವ್ಯತ್ಯಾಸಗಳಿವೆ
6,ಫ್ರಾಂಕೋ ಪರ್ಷಿಯ ಯುದ್ಧ
7,ಐದುನೂರು ಅಡಿಗಳಲ್ಲಿ
8,ಗ್ರಾಮೀಣ ಬಡ ಜನರಿಗಾಗಿ ಕಡಿಮೆ ಪ್ರೀಮಿಯಂ ಇರುವ ಆಕರ್ಷಕ ವಿಮಾ ಯೋಜನೆಗಳು
9,ಟ್ರಯಲ್ ನಿಬಂಧನೆಗಳು ಒಂದು ಪರ್ಯಾಯ ನಿಯಮವನ್ನೂ ಪರಿಚಯಿಸಿದೆ


Good! This looks better. We have removed most special characters from transcriptions and normalized them to lower-case only.

In CTC, it is common to classify speech chunks into letters, so we will do the same here. 
Let's extract all distinct letters of the training and test data and build our vocabulary from this set of letters.

We write a mapping function that concatenates all transcriptions into one long transcription and then transforms the string into a set of chars. 
It is important to pass the argument `batched=True` to the `map(...)` function so that the mapping function has access to all transcriptions at once.

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_test = test_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=test_dataset.column_names)

Now, we create the union of all distinct letters in the training dataset and test dataset and convert the resulting list into an enumerated dictionary.

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0])| set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'6': 0,
 '೨': 1,
 'ೇ': 2,
 'ಛ': 3,
 'ಿ': 4,
 'ಞ': 5,
 'ಊ': 6,
 '೧': 7,
 'ಾ': 8,
 'n': 9,
 "'": 10,
 'ಆ': 11,
 '೯': 12,
 '/': 13,
 '2': 14,
 'ಉ': 15,
 'ಂ': 16,
 '8': 17,
 'ಢ': 18,
 '೪': 19,
 '೮': 20,
 'ಲ': 21,
 'ೌ': 22,
 'ಙ': 23,
 'ವ': 24,
 'ಬ': 25,
 'ಧ': 26,
 'ಗ': 27,
 'ೋ': 28,
 'ಳ': 29,
 'e': 30,
 'u': 31,
 'ಥ': 32,
 'ಭ': 33,
 'ಚ': 34,
 '೩': 35,
 'ಖ': 36,
 '’': 37,
 'ಓ': 38,
 'ಠ': 39,
 'ಈ': 40,
 'ಕ': 41,
 'ಔ': 42,
 'ು': 43,
 ' ': 44,
 'ಋ': 45,
 'ತ': 46,
 'ಐ': 47,
 'ೀ': 48,
 'ೆ': 49,
 'ಮ': 50,
 'ಘ': 51,
 '೫': 52,
 'ೃ': 53,
 '\u200c': 54,
 'ೈ': 55,
 '೬': 56,
 '5': 57,
 'ಷ': 58,
 't': 59,
 '1': 60,
 '\u200d': 61,
 '_': 62,
 'ಏ': 63,
 'r': 64,
 'ದ': 65,
 'ರ': 66,
 'x': 67,
 'ಇ': 68,
 '್': 69,
 '7': 70,
 '೭': 71,
 '(': 72,
 'ಫ': 73,
 'ನ': 74,
 'ೊ': 75,
 'ೂ': 76,
 'ಅ': 77,
 'ಪ': 78,
 'ಝ': 79,
 ')': 80,
 'ಜ': 81,
 'ಒ': 82,
 'ಯ': 83,
 '೦': 84,
 '3': 85,
 '4': 86,
 'l': 87,
 'ಶ': 88,
 'ಣ': 89,
 'ಟ': 90,
 'ಎ': 91,
 'ಸ': 92,
 'ಹ': 93,
 'ಃ': 94,
 '0': 95,
 'i': 96,
 '9': 97,
 'ಡ': 98}

Cool, we see that all letters of the Devanagri occur in the dataset (which is not really surprising) and we also extracted the special character `" "`. Note that we did not exclude those special characters because: 

- The model has to learn to predict when a word is finished or else the model prediction would always be a sequence of chars which would make it impossible to separate words from each other.

One should always keep in mind that the data-preprocessing is a very important step before training your model. E.g., we don't want our model to differentiate between `a` and `A` just because we forgot to normalize the data. The difference between `a` and `A` does not depend on the "sound" of the letter at all, but more on grammatical rules - *e.g.* use a capitalized letter at the beginning of the sentence. So it is sensible to remove the difference between capitalized and non-capitalized letters so that the model has an easier time learning to transcribe speech. 

It is always advantageous to get help from a native speaker of the language you would like to transcribe to verify whether the assumptions you made are sensible.

To make it clearer that `" "` has its own token class, we give it a more visible character `|`. In addition, we also add an "unknown" token so that the model can later deal with characters not encountered in training set. 

Finally, we also add a padding token that corresponds to CTC's "*blank token*". The "blank token" is a core component of the CTC algorithm. For more information, please take a look at the "Alignment" section [here](https://distill.pub/2017/ctc/).

In [ ]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [ ]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)

101

Cool, now our vocabulary is complete and consists of 101 tokens, which means that the linear layer that we will add on top of the pretrained XLSR-Wav2Vec2 checkpoint will have an output dimension of 101.

Let's now save the vocabulary as a json file.

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In a final step, we use the json file to instantiate an object of the `Wav2Vec2CTCTokenizer` class.

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Next, we will create the feature extractor.

### Create XLSR-Wav2Vec2 Feature Extractor

Speech is a continuous signal and to be treated by computers, it first has to be discretized, which is usually called **sampling**. The sampling rate hereby plays an important role in that it defines how many data points of the speech signal are measured per second. Therefore, sampling with a higher sampling rate results in a better approximation of the *real* speech signal but also necessitates more values per second.

A pretrained checkpoint expects its input data to have been sampled more or less from the same distribution as the data it was trained on. The same speech signals sampled at two different rates have a very different distribution, *e.g.*, doubling the sampling rate results in data points being twice as long. Thus, 
before fine-tuning a pretrained checkpoint of an ASR model, it is crucial to verify that the sampling rate of the data that was used to pretrain the model matches the sampling rate of the dataset used to fine-tune the model.

XLSR-Wav2Vec2 was pretrained on the audio data of [Babel](https://huggingface.co/datasets/librispeech_asr), 
[Multilingual LibriSpeech (MLS)](https://ai.facebook.com/blog/a-new-open-data-set-for-multilingual-speech-research/), and [Common Voice](https://huggingface.co/datasets/common_voice). Most of those datasets were sampled at 16kHz, so that Common Voice, sampled at 48kHz, has to be downsampled to 16kHz for training. Therefore, we will have to downsample our fine-tuning data to 16kHz in the following.



A XLSR-Wav2Vec2 feature extractor object requires the following parameters to be instantiated:

- `feature_size`: Speech models take a sequence of feature vectors as an input. While the length of this sequence obviously varies, the feature size should not. In the case of Wav2Vec2, the feature size is 1 because the model was trained on the raw speech signal ${}^2$.
- `sampling_rate`: The sampling rate at which the model is trained on.
- `padding_value`: For batched inference, shorter inputs need to be padded with a specific value
- `do_normalize`: Whether the input should be *zero-mean-unit-variance* normalized or not. Usually, speech models perform better when normalizing the input
- `return_attention_mask`: Whether the model should make use of an `attention_mask` for batched inference. In general, XLSR-Wav2Vec2 models should **always** make use of the `attention_mask`.

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

Great, XLSR-Wav2Vec2's feature extraction pipeline is thereby fully defined!

To make the usage of XLSR-Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are *wrapped* into a single `Wav2Vec2Processor` class so that one only needs a `model` and `processor` object.

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

If one wants to re-use the just created processor and the fine-tuned model of this notebook, one can mount his/her google drive to the notebook and save all relevant files there. To do so, please uncomment the following lines. 

We will give the fine-tuned model the name `"wav2vec2-large-xlsr-kn"`.

In [ ]:
processor.save_pretrained("container_0/wav2vec2-large-xlsr-kn")

Next, we can prepare the dataset.

### Preprocess Data

So far, we have not looked at the actual values of the speech signal but just kept the path to its file in the dataset. `XLSR-Wav2Vec2` expects the audio file in the format of a 1-dimensional array, so in the first step, let's load all audio files into the dataset object.


The audio is present in `.wav` format. We use `torchaudio` here to load the file.

An audio file usually stores both its values and the sampling rate with which the speech signal was digitalized. We want to store both in the dataset and write a `map(...)` function accordingly.

In [ ]:
import torchaudio

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = speech_array[0].numpy()
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

/opt/conda/lib/python3.7/site-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '


In [ ]:
train_dataset = train_dataset.map(speech_file_to_array_fn, remove_columns=train_dataset.column_names)

In [ ]:
test_dataset = test_dataset.map(speech_file_to_array_fn, remove_columns=test_dataset.column_names)

In [ ]:
train_dataset["sampling_rate"][:10]

[48000, 48000, 48000, 48000, 48000, 48000, 48000, 48000, 48000, 48000]

Great, now we've successfully read in all the audio files, but since we know that data is sampled at 48kHz, we need to resample the audio files to 16kHz. 

Let's make use of the [`librosa`](https://github.com/librosa/librosa) library to downsample the data.

In [ ]:
import librosa
import numpy as np

def resample(batch):
    batch["speech"] = librosa.resample(np.asarray(batch["speech"]), 48_000, 16_000)
    batch["sampling_rate"] = 16_000
    return batch

In [ ]:
train_dataset = train_dataset.map(resample)
test_dataset = test_dataset.map(resample)

This seemed to have worked! Let's listen to a couple of audio files to better understand the dataset and verify that the audio was correctly loaded. 

**Note**: *You can click the following cell a couple of times to listen to different speech samples.*

In [ ]:
import IPython.display as ipd
import numpy as np
import random

rand_int = random.randint(0, len(train_dataset))

ipd.Audio(data=np.asarray(train_dataset[rand_int]["speech"]), autoplay=True, rate=16000)

It can be heard, that the speakers change along with their speaking rate, accent, and background environment, etc. Overall, the recordings sound acceptably clear though, which is to be expected from a crowd-sourced read speech corpus.

Let's do a final check that the data is correctly prepared, by printing the shape of the speech input, its transcription, and the corresponding sampling rate.

**Note**: *You can click the following cell a couple of times to verify multiple samples.*

In [ ]:
# rand_int = random.randint(0, len(common_voice_train))

print("Target text:", train_dataset[rand_int]["target_text"])
print("Input array shape:", np.asarray(train_dataset[rand_int]["speech"]).shape)
print("Sampling rate:", train_dataset[rand_int]["sampling_rate"])

Target text: ಹೀಗೆ ಮಾಡುವುದರಿಂದ ದುರ್ಬಲ ಸ್ಥಾನಗಳಿಲ್ಲದೆ ಮೇಲಿನ ಮೈ ಎಲ್ಲವೂ ಸರಿಯಾಗಿ ಅಂಟಿಕೊಳ್ಳುವುದು 
Input array shape: (128342,)
Sampling rate: 16000


Good! Everything looks fine - the data is a 1-dimensional array, the sampling rate always corresponds to 16kHz, and the target text is normalized.

Finally, we can process the dataset to the format expected by the model for training. We will again make use of the `map(...)` function.

First, we check that the data samples have the same sampling rate of 16kHz.
Second, we extract the `input_values` from the loaded audio file. In our case, this includes only normalization, but for other speech models, this step could correspond to extracting, *e.g.* [Log-Mel features](https://en.wikipedia.org/wiki/Mel-frequency_cepstrum). 
Third, we encode the transcriptions to label ids.

**Note**: This mapping function is a good example of how the `Wav2Vec2Processor` class should be used. In "normal" context, calling `processor(...)` is redirected to `Wav2Vec2FeatureExtractor`'s call method. When wrapping the processor into the `as_target_processor` context, however, the same method is redirected to `Wav2Vec2CTCTokenizer`'s call method.
For more information please check the [docs](https://huggingface.co/transformers/master/model_doc/wav2vec2.html#transformers.Wav2Vec2Processor.__call__).

In [ ]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [ ]:
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names, batch_size=8, num_proc=1, batched=True)
test_dataset = test_dataset.map(prepare_dataset, remove_columns=test_dataset.column_names, batch_size=8, num_proc=1, batched=True)

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

Next, the evaluation metric is defined. As mentioned earlier, the 
predominant metric in ASR is the word error rate (WER), hence we will use it in this notebook as well.

In [ ]:
from datasets import load_metric

In [ ]:
wer_metric = load_metric("wer")

The model will return a sequence of logit vectors:
$\mathbf{y}_1, \ldots, \mathbf{y}_m$ with $\mathbf{y}_1 = f_{\theta}(x_1, \ldots, x_n)[0]$ and $n >> m$.

A logit vector $\mathbf{y}_1$ contains the log-odds for each word in the vocabulary we defined earlier, thus $\text{len}(\mathbf{y}_i) =$ `config.vocab_size`. We are interested in the most likely prediction of the model and thus take the `argmax(...)` of the logits. Also, we transform the encoded labels back to the original string by replacing `-100` with the `pad_token_id` and decoding the ids while making sure that consecutive tokens are **not** grouped to the same token in CTC style ${}^1$.

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

Now, we can load the pretrained `XLSR-Wav2Vec2` checkpoint. The tokenizer's `pad_token_id` must be to define the model's `pad_token_id` or in the case of `Wav2Vec2ForCTC` also CTC's *blank token* ${}^2$. To save GPU memory, we enable PyTorch's [gradient checkpointing](https://pytorch.org/docs/stable/checkpoint.html) and also set the loss reduction to "*mean*".

Because the dataset is quite small (~6h of training data) and because Common Voice is quite noisy, fine-tuning Facebook's [wav2vec2-large-xlsr-53 checkpoint](https://huggingface.co/facebook/wav2vec2-large-xlsr-53) seems to require some hyper-parameter tuning. Therefore, I had to play around a bit with different values for dropout, [SpecAugment](https://arxiv.org/abs/1904.08779)'s masking dropout rate, layer dropout, and the learning rate until training seemed to be stable enough. 

**Note**: When using this notebook to train XLSR-Wav2Vec2 on another language of Common Voice those hyper-parameter settings might not work very well. Feel free to adapt those depending on your use case. 

In [ ]:
from transformers import Wav2Vec2ForCTC
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore. 
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [ ]:
model.freeze_feature_extractor()

In a final step, we define all parameters related to training. 
To give more explanation on some of the parameters:
- `group_by_length` makes training more efficient by grouping training samples of similar input length into one batch. This can significantly speed up training time by heavily reducing the overall number of useless padding tokens that are passed through the model
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
  output_dir="container_0/ckpts/",
  logging_dir = "container_0/runs/",
  group_by_length=True,
  per_device_train_batch_size=8,
  per_device_eval_batch_size=8,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=200,
  learning_rate=4e-4,
  warmup_steps=int(0.1*1320), #10%
  save_total_limit=2,
)

Now, all instances can be passed to Trainer and we are ready to start training!

In [ ]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor.feature_extractor,
)



---

${}^1$ To allow models to become independent of the speaker rate, in CTC, consecutive tokens that are identical are simply grouped as a single token. However, the encoded labels should not be grouped when decoding since they don't correspond to the predicted tokens of the model, which is why the `group_tokens=False` parameter has to be passed. If we wouldn't pass this parameter a word like `"hello"` would incorrectly be encoded, and decoded as `"helo"`.

${}^2$ The blank token allows the model to predict a word, such as `"hello"` by forcing it to insert the blank token between the two l's. A CTC-conform prediction of `"hello"` of our model would be `[PAD] [PAD] "h" "e" "e" "l" "l" [PAD] "l" "o" "o" [PAD]`.

### Training

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (*right mouse click -> inspect -> Console tab and insert code*).

```javascript
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}
setInterval(ConnectButton,60000);
```

In [ ]:
trainer.train()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /workspace/.netrc
wandb: wandb version 0.10.23 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
200,8.824500,3.603292,1.000000,70.919800,6.204000
400,3.528700,3.452219,1.000000,70.960200,6.201000
600,3.014500,1.026335,0.973120,71.779400,6.130000
800,0.809200,0.473118,0.697280,72.626100,6.058000
1000,0.537800,0.378213,0.620480,72.618100,6.059000
1200,0.414500,0.315756,0.553280,73.113400,6.018000
1400,0.348900,0.301182,0.519360,73.292200,6.003000
1600,0.299500,0.290357,0.493760,74.332800,5.919000
1800,0.257500,0.283907,0.481280,74.281000,5.923000
2000,0.231700,0.286065,0.493440,74.057700,5.941000


TrainOutput(global_step=7410, training_loss=0.5628429738535733, metrics={'train_runtime': 25342.5891, 'train_samples_per_second': 0.292, 'total_flos': 2.56750694618022e+19, 'epoch': 30.0, 'init_mem_cpu_alloc_delta': 9868187, 'init_mem_gpu_alloc_delta': 1262169088, 'init_mem_cpu_peaked_delta': 18306, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 52373522, 'train_mem_gpu_alloc_delta': 3786999808, 'train_mem_cpu_peaked_delta': 193443886, 'train_mem_gpu_peaked_delta': 6116016128})

The training loss goes down and we can see that the WER on the test set also improves nicely. Because this notebook is just for demonstration purposes, we can stop here.

The resulting model of this notebook has been saved.

Lastly, we save the final model :)

In [ ]:
trainer.save_model("container_0/wav2vec2-large-xlsr-kn")

# Usage

In [ ]:
import torch
import torchaudio
from datasets import load_dataset
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor

# test_dataset = #TODO: WRITE YOUR CODE TO LOAD THE TEST DATASET. For sample see the Colab link in Training Section.

processor = Wav2Vec2Processor.from_pretrained("container_0/wav2vec2-large-xlsr-kn")
model = Wav2Vec2ForCTC.from_pretrained("container_0/wav2vec2-large-xlsr-kn")

resampler = torchaudio.transforms.Resample(48_000, 16_000) # The original data was with 48,000 sampling rate. You can change it according to your input.

# Preprocessing the datasets.
# We need to read the audio files as arrays
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)
inputs = processor(test_dataset["speech"][:2], sampling_rate=16_000, return_tensors="pt", padding=True)

with torch.no_grad():
    logits = model(inputs.input_values, attention_mask=inputs.attention_mask).logits

predicted_ids = torch.argmax(logits, dim=-1)

print("Prediction:", processor.batch_decode(predicted_ids))
print("Reference:", test_dataset["text"][:2])

# Evaluation

In [ ]:
import torch
import torchaudio
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import re

# test_dataset = #TODO: WRITE YOUR CODE TO LOAD THE TEST DATASET. For sample see the Colab link in Training Section.
test_dataset = dataset['test'].map(remove_special_characters)

wer = load_metric("wer")

processor = Wav2Vec2Processor.from_pretrained("container_0/wav2vec2-large-xlsr-kn")
model = Wav2Vec2ForCTC.from_pretrained("container_0/wav2vec2-large-xlsr-kn")
model.to("cuda")

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\–\…]'
resampler = torchaudio.transforms.Resample(48_000, 16_000)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def speech_file_to_array_fn(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    speech_array, sampling_rate = torchaudio.load(batch["path"])
    batch["speech"] = resampler(speech_array).squeeze().numpy()
    return batch

test_dataset = test_dataset.map(speech_file_to_array_fn)

# Preprocessing the datasets.
# We need to read the aduio files as arrays
def evaluate(batch):
    inputs = processor(batch["speech"], sampling_rate=16_000, return_tensors="pt", padding=True)
    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits
        pred_ids = torch.argmax(logits, dim=-1)
        batch["pred_strings"] = processor.batch_decode(pred_ids)
        return batch

result = test_dataset.map(evaluate, batched=True, batch_size=8)
# NOTE : The original test set was different and this result may be skewed. The original test set gave around 14.53% WER.
print("WER: {:2f}".format(100 * wer.compute(predictions=result["pred_strings"], references=result["text"])))

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.



WER: 27.086665


## Test with your own Audio

Thanks to [Eric Liam](https://huggingface.co/voidful) for sharing his notebook on which this is based. 

In [ ]:
%%capture
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install ffmpeg-python

In [ ]:
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);

function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

In [ ]:
pip install Js2Py

In [ ]:
from IPython.display import HTML, Audio
from js2py import eval_js
from base64 import b64decode
import wave
from scipy.io.wavfile import read as wav_read
import io
import numpy as np
import ffmpeg

def write_wav(f, sr, x, normalized=False):
    f = wave.open(f, "wb")
    f.setnchannels(1)
    f.setsampwidth(2)
    f.setframerate(sr)
    
    wave_data = x.astype(np.short)
    f.writeframes(wave_data.tobytes())
    f.close()

def get_audio():
  global hnum

  # call microphone
  display(HTML(AUDIO_HTML))
  data = eval_js('data')
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
      .input('pipe:0')
      .output('pipe:1', format='wav')
      .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]
  sr, audio = wav_read(io.BytesIO(riff))
  # save
  human_sound_file = "demo.wav"
  write_wav(human_sound_file, sr, audio)

  return human_sound_file

In [ ]:
import torchaudio
from datasets import load_dataset, load_metric
from transformers import (
    Wav2Vec2ForCTC,
    Wav2Vec2Processor,
    AutoTokenizer, 
    AutoModelWithLMHead 
)
import torch
import re
import sys

model_name = "container_0/wav2vec2-large-xlsr-kn"
device = "cuda"
processor_name = "container_0/wav2vec2-large-xlsr-kn"

chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�\–\…]'

model = Wav2Vec2ForCTC.from_pretrained(model_name).to(device)
processor = Wav2Vec2Processor.from_pretrained(processor_name)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
resampler = torchaudio.transforms.Resample(orig_freq=48_000, new_freq=16_000)

def load_file_to_data(file):
    batch = {}
    speech, _ = torchaudio.load(file)
    batch["speech"] = resampler.forward(speech.squeeze(0)).numpy()
    batch["sampling_rate"] = resampler.new_freq
    return batch

def predict(data):
    features = processor(data["speech"], sampling_rate=data["sampling_rate"], padding=True, return_tensors="pt")
    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)
    with torch.no_grad():
        logits = model(input_values, attention_mask=attention_mask).logits
    
    decoded_results = []
    for logit in logits:
        pred_ids = torch.argmax(logit, dim=-1)
        decoded_results.append(processor.decode(pred_ids))
    return decoded_results


In [ ]:
get_audio()

In [ ]:
predict(load_file_to_data('./demo.wav'))

This means "Thank You" :)